# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_MINIMAL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Design the database using Apache Cassandra according to the queries.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### In order to query the results about songs according to the sessionId and itemInSession, the Primary key is composed of Partition Key: session_id and Clustering Key: item_in_session. The Clustering key is used to make sure that the data inside the table is ordered by the item_in_session. 



In [8]:
query1 = """CREATE TABLE IF NOT EXISTS songs (
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (session_id, item_in_session)
            )"""
try:
    session.execute(query1)
except Exception as e:
    print(e)
                    

#### Select the value of session_id, item_in_session, artist, song, length from the event_datafile_new.csv, then insert into the songs table created before.

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query_1 = """SELECT artist, song, length FROM songs WHERE session_id = 338 AND item_in_session = 4"""
try:
    results = session.execute(query_1)
except Exception as e:
    print(e)


df = pd.DataFrame([[row.artist, row.song] for row in results], columns=["artist", "song"])
print(df)

      artist                             song
0  Faithless  Music Matters (Mark Knight Dub)


### 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### In order to query the results about songs according to the userid and sessionid, the Partition Key: session_id and Clustering Key: item_in_session and sessionid. The Clustering key:itemInSession is used to make sure that the data inside the table is ordered by the item_in_session. 


In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = """CREATE TABLE IF NOT EXISTS user_sessions (
            user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, first_name TEXT, last_name TEXT, 
            PRIMARY KEY ((user_id, session_id), item_in_session)
            )"""

try:
    session.execute(query2)
except Exception as e:
    print(e)
                    

#### Select the value of user_id, session_id, item_in_session, artist, song, first_name, last_name from the event_datafile_new.csv, then insert into the user_sessions table created before.

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_sessions (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [16]:
## Add in the SELECT statement to verify the data was entered into the table
query_2 = """SELECT artist, song FROM user_sessions WHERE user_id = 10 AND session_id = 182"""
try:
    results = session.execute(query_2)
except Exception as e:
    print(e)

df = pd.DataFrame([[row.artist, row.song] for row in results], columns=["artist", "song"])
print(df)

              artist                                               song
0   Down To The Bone                                 Keep On Keepin' On
1       Three Drives                                        Greece 2000
2  Sebastien Tellier                                          Kilometer
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...


### 3. Query every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' 

#### song is the Partition Key and user_id is the Partition Key. song should be before user_id because song the first we need to query and user_id is the clustering column. (firs)

In [18]:
query3  = """CREATE TABLE IF NOT EXISTS users_new (
             song TEXT,
             user_id INT,
             first_name TEXT,
             last_name TEXT,
             PRIMARY KEY (song, user_id))"""
try:
    session.execute(query3)
except Exception as e:
    print(e)
                    

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO users_new (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [22]:
## Add in the SELECT statement to verify the data was entered into the table
query_3 = """SELECT first_name, last_name FROM users_new WHERE song = 'All Hands Against His Own'"""
try:
    results = session.execute(query_3)
except Exception as e:
    print(e)

df = pd.DataFrame([[row.first_name, row.last_name] for row in results], columns=["first_name", "last_name"] )
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [23]:
# Drop the table before closing out the sessions
drop_songs = "DROP TABLE IF EXISTS songs"
drop_user_sessions = "DROP TABLE IF EXISTS user_sessions"
drop_users_new = "DROP TABLE IF EXISTS users_new"
try:
    session.execute(drop_songs)
    session.execute(drop_user_sessions)
    session.execute(drop_users_new)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()